## Расчет расстояния до адреса до границы

### Скачиваем библиотеки

In [ ]:
!pip install geopy
!pip install tqdm

### Импортируем библиотеки

In [1]:
import pandas as pd
from geopy.distance import geodesic as GD 
import json
from tqdm import tqdm

### Ипортируем данные с координатами

In [11]:
with open("data/adress_coord.json", 'r', encoding='utf-8-sig') as ad_cor:
    adress_coord_dict = json.load(ad_cor)
    
border_coord_df = pd.read_json("data//border.json")

In [12]:
border_coord_df.head()

,lon,lat
11,39.2300,49.8384
12,39.2218,49.8444
13,39.2224,49.8498
14,39.2136,49.8503
15,39.2142,49.8557


### Для удобства словарь с координатами адресов преобразуем в df

In [13]:
atress_coord_df = pd.DataFrame(adress_coord_dict.items(), columns=['adress', 'coord'])

In [14]:
atress_coord_df

,adress,coord
0,"Белгородская область, г. Белгород, ул. Серафим...","50.587197, 36.620164"
1,"Белгородская область, г Белгорд, ул имени Шуми...","50.565706, 36.604434"
2,"Белгородская область, г Белгород, ул генерала-...","50.595414, 36.587277"
3,"307170, Курская область, г. Железногорск, ул. ...","52.339203, 35.350871"
4,курская обл. п. Тим ул. Юбилейная д.34,"51.627241, 37.129500"
...,...,...
496,"Курская область, п. Черемушки 23 кв 19 обл, Ку...","51.604682, 36.255619"
497,"Белгородская обл., Белгородский р-н, с. Севрюк...","50.631932, 36.741526"
498,"Курская обл., г. Курск, пер. 2-й Стрелецкий, д...","51.727455, 36.214979"
499,"Курская обл., г. Курск, проспект Вячеслава Клы...","51.726205, 36.140239"


### Преобразуем координаты границы в список

In [15]:
list_border_coord = list([(row['lat'], row['lon']) for index, row in border_coord_df.iterrows()])

In [16]:
list_border_coord[0:10]

[(49.8384, 39.23),
 (49.8444, 39.2218),
 (49.8498, 39.2224),
 (49.8503, 39.2136),
 (49.8557, 39.2142),
 (49.8617, 39.2059),
 (49.8622, 39.1971),
 (49.8676, 39.1976),
 (49.8735, 39.1893),
 (49.8794, 39.1865)]

In [17]:
def distance_calculation(start_coord):
    list_dist = []
    for bord_coord in list_border_coord:
        #Для расчета расстояния используем функцию GD([1 координата точки],[2 координата точки].[единица измерения расстояния])
        dist = GD(start_coord, bord_coord).km
        #Добовляем в список результат
        list_dist.append(dist)
    
    #Возвращаем минимальную дистанцию из списке
    return min(list_dist)

In [18]:
tqdm.pandas()
atress_coord_df['dist_to_bor'] = atress_coord_df['coord'].progress_apply(lambda x: distance_calculation(x))

100%|████████████████████████████████████████████████████████████████████████████████| 501/501 [02:10<00:00,  3.84it/s]


### Просматриваем и проверяем результат

In [19]:
atress_coord_df

,adress,coord,dist_to_bor
0,"Белгородская область, г. Белгород, ул. Серафим...","50.587197, 36.620164",30.275355
1,"Белгородская область, г Белгорд, ул имени Шуми...","50.565706, 36.604434",27.640944
2,"Белгородская область, г Белгород, ул генерала-...","50.595414, 36.587277",30.340558
3,"307170, Курская область, г. Железногорск, ул. ...","52.339203, 35.350871",85.145191
4,курская обл. п. Тим ул. Юбилейная д.34,"51.627241, 37.129500",134.815118
...,...,...,...
496,"Курская область, п. Черемушки 23 кв 19 обл, Ку...","51.604682, 36.255619",84.349008
497,"Белгородская обл., Белгородский р-н, с. Севрюк...","50.631932, 36.741526",33.966367
498,"Курская обл., г. Курск, пер. 2-й Стрелецкий, д...","51.727455, 36.214979",92.868262
499,"Курская обл., г. Курск, проспект Вячеслава Клы...","51.726205, 36.140239",89.727109
